In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd

from beamngpy import BeamNGpy, Vehicle, Scenario, setup_logging
from beamngpy.sensors import Electrics, Camera, Damage, Timer

In [3]:
setup_logging()

# create BeamNG simulator instance
beamng = BeamNGpy(host='localhost', port=64256)

# create a scenario
# TODO: what other levels are possible?
scenario = Scenario(level='west_coast_usa', name='vehicle_state_scenario')

# create a vehicle, attach sensors
# TODO: use random UUID as vid
vehicle = Vehicle(vid='auto', model='etk800', licence='KASPAR', color='Green')

# set up sensors
pos = (-0.3, 1, 1.0)
direction = (0, 1, 0)
fov = 60
resolution = (512, 512 // 2)
front_camera = Camera(pos, direction, fov, resolution, colour=True, depth=False, annotation=False)

electrics = Electrics()
damage = Damage()
timer = Timer()

vehicle.attach_sensor('front_cam', front_camera)
vehicle.attach_sensor('electrics', electrics)
vehicle.attach_sensor('damage', damage)
vehicle.attach_sensor('timer', timer)

scenario.add_vehicle(vehicle, pos=(-717.121, 101, 118.675), rot=(0, 0, 45))

# compile scenario and produce scenario file
scenario.make(beamng)

2020-01-17 20:58:44,059 INFO     Started BeamNGpy logging.
2020-01-17 20:58:44,074 DEBUG    Bound memory for colour: 6688.auto.front_cam.colour
2020-01-17 20:58:44,076 DEBUG    Bound memory for depth: 6688.auto.front_cam.depth
2020-01-17 20:58:44,079 DEBUG    Bound memory for annotation: 6688.auto.front_cam.annotate


In [4]:
binary = beamng.determine_binary()


2020-01-17 20:58:48,218 DEBUG    Determined BeamNG.* binary to be: C:\Users\kaspa\Desktop\research-unlimited\trunk\Bin64\BeamNG.research.x64.exe


In [5]:
bng = beamng.open(launch=True)
bng.load_scenario(scenario)
bng.start_scenario()

2020-01-17 20:58:50,700 INFO     Opening BeamNPy instance...
2020-01-17 20:58:50,711 INFO     Started BeamNGpy server on localhost:64256
2020-01-17 20:58:50,714 DEBUG    Determined BeamNG.* binary to be: C:\Users\kaspa\Desktop\research-unlimited\trunk\Bin64\BeamNG.research.x64.exe
2020-01-17 20:58:50,720 DEBUG    Starting BeamNG process...
2020-01-17 20:58:59,031 DEBUG    Connection established. Awaiting "hello"...
2020-01-17 20:58:59,037 INFO     Started BeamNGpy communicating on ('127.0.0.1', 50021)
2020-01-17 20:59:28,396 DEBUG    Starting vehicle server for V:auto on: localhost:64257


In [6]:
NUM_OF_CAPTURES = 200
PERIOD = 0.2 # 200 ms

position = []
direction = []
velocity = []
wheel_speed = []
throttle = []
brake = []
steering = []
timestamp = []

IMAGE_STORE = 'C:\\Users\\kaspa\\Documents\\BeamNG.research\\screenshots\\capture'

for index in range(NUM_OF_CAPTURES):
    time.sleep(PERIOD)
    vehicle.update_vehicle()

    position.append(vehicle.state['pos'])
    direction.append(vehicle.state['dir'])
    velocity.append(3.6 * np.linalg.norm(vehicle.state['vel'])) # km/h

    sensors = bng.poll_sensors(vehicle)
    t = sensors['timer']['time']
    timestamp.append(t)
    wheel_speed.append(sensors['electrics']['values']['wheelspeed'])
    throttle.append(sensors['electrics']['values']['throttle'])
    brake.append(sensors['electrics']['values']['brake'])
    steering.append(sensors['electrics']['values']['steering_input'])

    sensors['front_cam']['colour'].save(f'{IMAGE_STORE}\\{index}.png', 'png')    

In [7]:
position = np.asarray(position)
direction = np.asarray(direction)

data = list(zip(
    timestamp, 
    position[:, 0], 
    position[:, 1], 
    position[:, 2], 
    direction[:, 0], 
    direction[:, 1], 
    direction[:, 2], 
    velocity, 
    wheel_speed, 
    throttle, 
    brake, 
    steering)
)
columns = [
    'timestamp',
    'position_x',
    'position_y',
    'position_z',
    'direction_x',
    'direction_y',
    'direction_z',
    'velocity',
    'wheel_speed',
    'throttle',
    'brake',
    'steering'
]
df = pd.DataFrame(data, columns=columns)
df

,timestamp,position_x,position_y,position_z,direction_x,direction_y,direction_z,velocity,wheel_speed,throttle,brake,steering
0,3.171000,-717.133057,100.987595,118.545441,-0.706914,-0.707268,-0.006672,0.000464,0.000018,0.000035,0.000000,0.000000
1,3.821000,-717.133118,100.987595,118.545441,-0.706914,-0.707267,-0.006672,0.000590,0.000013,0.000035,0.000000,0.000000
2,4.371000,-717.133057,100.987595,118.545441,-0.706915,-0.707267,-0.006673,0.000278,0.000003,0.000035,0.007141,0.000000
3,4.921000,-717.133118,100.987595,118.545441,-0.706914,-0.707268,-0.006674,0.000154,0.000016,0.000035,0.453750,0.000000
4,5.421000,-717.133118,100.987595,118.545448,-0.706914,-0.707268,-0.006674,0.000663,0.000122,0.000035,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
195,93.621001,-400.868774,327.036804,101.995827,0.998588,-0.052879,-0.005183,6.992370,1.730087,0.196831,0.012490,-0.520494
196,94.121001,-399.603760,327.285400,102.005989,0.995450,0.094560,0.011707,13.301546,3.900211,0.004959,0.012490,-0.654570
197,94.621001,-397.961945,328.052216,101.996101,0.938367,0.345571,-0.006994,11.869047,3.123664,0.000000,0.013841,-0.633919
198,95.071002,-396.914612,328.883301,101.996719,0.856189,0.516636,-0.005243,9.538798,2.601724,0.000000,0.013841,-0.414800


In [13]:
df.to_csv(f'{IMAGE_STORE}\\data.csv', encoding='utf-8', index=False)

In [73]:
!cd C:\Users\kaspa\\Documents\BeamNG.research\screenshots\capture
!dir

Volume in drive C has no label.
 Volume Serial Number is 16C5-554F

 Directory of C:\Users\kaspa\Documents\BeamNG.research\screenshots

17.01.2020  19:36    <DIR>          .
17.01.2020  19:36    <DIR>          ..
17.01.2020  00:37    <DIR>          2020-01
17.01.2020  21:17    <DIR>          capture
               0 File(s)              0 bytes
               4 Dir(s)  458˙837˙217˙280 bytes free


In [53]:
bng.close()

2020-01-17 00:02:15,972 INFO     Closing BeamNGpy instance...
2020-01-17 00:02:15,974 INFO     send #0: 64256
2020-01-17 00:02:15,986 INFO     send #1: {'type': 'CloseShmem', 'name': '5432.auto.front_cam.colour'}
2020-01-17 00:02:15,987 INFO     send #2: 0000000000000049
2020-01-17 00:02:15,988 INFO     send #3: b'\x82\xa4type\xaaCloseShmem\xa4name\xba5432.auto.front_cam.colour'
2020-01-17 00:02:15,989 INFO     recv #0: 64256
2020-01-17 00:02:16,007 INFO     recv #1: 18
2020-01-17 00:02:16,009 INFO     recv #2: b'\x81\xa4type\xabClosedShmem'
2020-01-17 00:02:16,018 INFO     recv #3: b''
2020-01-17 00:02:16,020 INFO     recv #4: {'type': 'ClosedShmem'}
2020-01-17 00:02:16,021 INFO     send #0: 64256
2020-01-17 00:02:16,022 INFO     send #1: {'type': 'CloseShmem', 'name': '5432.auto.front_cam.depth'}
2020-01-17 00:02:16,023 INFO     send #2: 0000000000000048
2020-01-17 00:02:16,024 INFO     send #3: b'\x82\xa4type\xaaCloseShmem\xa4name\xb95432.auto.front_cam.depth'
2020-01-17 00:02:16,02